In [ ]:
import numpy as np
import pandas as pd


from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
from keras.utils import to_categorical

import matplotlib.pyplot as plt
from sklearn import metrics

### Loading the data from csv files

In [ ]:
img_size = 32

# Load training data from csv file
data = pd.read_csv("./char_train.csv")
data_t = pd.read_csv("./char_test.csv")

# Extract feature columns
feature_cols = list(data.columns[1:])
feature_cols_t = list(data_t.columns[1:])

# Extract target column 'label'
target_col = data.columns[0]
target_col_t = data_t.columns[0]

# Separate the data into feature data and target data (X and y, respectively)
X = data[feature_cols]
y = data[target_col]
X_t = data_t[feature_cols_t]
y_t = data_t[target_col_t]


In [ ]:
X = X.values.reshape(X.shape[0], img_size, img_size, 1)
X_t = X_t.values.reshape(X_t.shape[0], img_size, img_size, 1)

In [ ]:
y = to_categorical(y)
y_t = to_categorical(y_t)

In [ ]:

img_size = 32
X = np.array(X,dtype='float32')
X = X.astype('float32')
y = y.astype('int')

### Training the models with 5-fold cross validation

In [ ]:

stratkf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


models = []

for train_index, val_index in stratkf.split(X, np.argmax(y, axis=1)):
    X_train_fold, X_val_fold = X[train_index], X[val_index]
    Y_train_fold, Y_val_fold = y[train_index], y[val_index]
    
    # Create the model
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))  # Add more convolutional layers
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))  # Increase the number of neurons in Dense layers
    model.add(Dropout(0.5))
    model.add(Dense(52, activation='softmax'))

    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    
    
    model.fit(X_train_fold, Y_train_fold, epochs=15, batch_size=100, validation_data=(X_val_fold, Y_val_fold))
    
    score = model.evaluate(X_val_fold, Y_val_fold, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    
    models.append(model)

### Checking the accuracy

In [ ]:

Y_pred = []

for model in models:
    y_pred = model.predict(X_t)
    Y_pred.append(y_pred)
    

Y_pred = np.array(Y_pred)
Y_pred = np.argmax(Y_pred, axis=2)


Y_pred = np.transpose(Y_pred)

Y_pred = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=1, arr=Y_pred)

print("Accuracy: ", accuracy_score(np.argmax(y_t, axis=1), Y_pred))

### Saving models locally

In [ ]:
for i, model in enumerate(models):
    model.save("./models/model_"+str(i)+".h5")

In [ ]:
loaded_model = []

for i in range(5):
    # Loading model
    model = load_model("./models/model_"+str(i)+".h5")
    loaded_model.append(model)

### Prediction and Generating Confusion Matrix

In [ ]:
def load_models():
    loaded_model = []

    for i in range(5):
        # Loading model
        model = load_model("./models/model_"+str(i)+".h5")
        loaded_model.append(model)
    return loaded_model

def predict(models, X_t):
    Y_pred = []

    for model in models:
        y_pred = model.predict(X_t)
        Y_pred.append(y_pred)


    Y_pred = np.array(Y_pred)
    Y_pred = np.argmax(Y_pred, axis=2)


    Y_pred = np.transpose(Y_pred)

    Y_pred = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=1, arr=Y_pred)
    return Y_pred

In [ ]:
models = load_models()

In [ ]:
models

In [ ]:
Y_pred = predict(models, X_t)

confusion_matrix = metrics.confusion_matrix(np.argmax(y_t, axis=1), Y_pred)

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix).plot()

plt.show()